In [2]:
import torch

assert torch.__version__ >= "4.8.0", "This script requires PyTorch version 1.8.0 or higher."

AssertionError: This script requires PyTorch version 1.8.0 or higher.

!pip install einops

In [3]:
!pip install einops

In [1]:
import torch
import torch.nn as nn
from einops import rearrange, repeat
from einops.layers.torch import Rearrange

# helpers

def pair(t):
    return t if isinstance(t, tuple) else (t, t)

# classes



class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)


class TransformerBlock(nn.Module):
    def __init__(self, dim, heads, mlp_dim, dropout = 0.):
        super().__init__()
        self.dim = dim
        self.mlp_dim = mlp_dim
        self.attention = nn.MultiheadAttention(embed_dim=dim, num_heads=heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(dim)
        self.norm2 = nn.LayerNorm(dim)
        self.ff = FeedForward(dim, mlp_dim, dropout)

    def forward(self, x):
        x = self.norm1(x + self.attention(x, x, x)[0])
        x = self.norm2(x + self.ff(x))
        return x


class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0.):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers=nn.ModuleList([
                TransformerBlock(dim, heads, mlp_dim, dropout)

            ])
    def forward(self, x):
        return x

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, pool = 'cls', channels = 3, dim_head = 64, dropout = 0., emb_dropout = 0.):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

        num_patches = (image_height // patch_height) * (image_width // patch_width)
        patch_dim = channels * patch_height * patch_width
        assert pool in {'cls', 'mean'}, 'pool type must be either cls (class token) or mean (mean pooling)'

        self.to_patch_embedding = nn.Sequential(
            Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
            nn.Linear(patch_dim, dim),
        )

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.dropout = nn.Dropout(emb_dropout)

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim, dropout)

        self.pool = pool
        self.to_latent = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, num_classes)
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)
        b, n, _ = x.shape

        cls_tokens = repeat(self.cls_token, '() n d -> b n d', b = b)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding[:, :(n + 1)]
        x = self.dropout(x)

        x = self.transformer(x)

        x = x.mean(dim = 1) if self.pool == 'mean' else x[:, 0]

        x = self.to_latent(x)
        return self.mlp_head(x)

# Example Usage:
# This demonstrates how to instantiate and use the Vision Transformer model.

# Configuration for a 'base' ViT model
image_encoder = ViT(
    image_size = 224,
    patch_size = 32,
    num_classes = 768,  # This is the embedding dimension, not the number of classes for classification
    dim = 768,
    depth = 12,
    heads = 12,
    mlp_dim = 3072,
    dropout = 0.1,
    emb_dropout = 0.1
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_encoder.to(device)


# Create a dummy image tensor
dummy_image = torch.randn(10, 3, 224, 224)
dummy_image=dummy_image.to(device)

# Get the image embeddings
image_features = image_encoder(dummy_image)

print("Shape of the output image features:", image_features.shape)

Shape of the output image features: torch.Size([10, 768])


In [4]:
sequences = torch.tensor([
  [1, 2, 3, 4, 5],
  [6, 7, 8, 0, 0] 
])

mask= (sequences== 0).unsqueeze(1).unsqueeze(2)  # Create a mask for padding tokens

In [ ]:
print(sequences.shape)
print(mask)
print("Shape of the mask:", mask.shape)  # [Batcch, 1, 1, Sequence Length]

torch.Size([2, 5])
tensor([[[[False, False, False, False, False]]],


        [[[False, False, False,  True,  True]]]])
Shape of the mask: torch.Size([2, 1, 1, 5])


In [2]:
total_params = sum(p.numel() for p in image_encoder.parameters())
print("Total parameters:", total_params)

Total parameters: 10079232


In [12]:
from image_encoder import ViT

image_encoder =ViT(
    image_size = 224,
    patch_size = 16,
    num_classes = 512,  # This is the embedding dimension, not the number of classes for classification
    dim = 768,
    layer = 12,
    heads = 12,
    mlp_dim = 3072,
    dropout = 0.1,
    emb_dropout = 0.1
).to(device)



In [ ]:
import argparse


parser=argparse.ArgumentParser(description="I am Biggner with argparse")

parser.add_argument('number1', type=int, default=12, help='first number to add')
parser.add_argument("number2",type=int,default=13,help="second number to add")
parser.add_argument("operations",type=str, choices=['add', 'subtract', 'multiply', 'divide'], help="operation to perform")

args = parser.parse_args()


Total parameters: 8225792
